In [1]:
import os
import sys
sys.path.append(os.getcwd() + "/../")
from cubercnn import util, vis

import cv2 as cv
import numpy as np
import pickle


In [23]:
images_path = "/home/max/Documents/RA/egowholemocap/notebooks/unwrapped"
image_files = sorted(
    [f for f in os.listdir(images_path)
    if os.path.splitext(f)[1].lower() in (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")],
    key=lambda x: int(os.path.splitext(x)[0])
)

In [24]:
frame = cv.imread(os.path.join(images_path, image_files[0]))

h, w = frame.shape[:2]
        
focal_length_ndc = 4.0
focal_length = focal_length_ndc * h / 2

px, py = w/2, h/2

K = np.array([
            [focal_length, 0.0, px], 
            [0.0, focal_length, py], 
            [0.0, 0.0, 1.0]
        ])

In [27]:


with open("/home/max/Documents/RA/ovmono3d/output/insta360_unwrapped/detections_reordered.pkl", 'rb') as f:
        data = pickle.load(f)
        
frames_data = []
for frame_idx in range(len(data)):
        meshes = []
        frame_data = data[frame_idx]
        object_centers = frame_data['pred_center_cam']
        object_dimensions = frame_data['pred_dimensions']
        object_rotations = frame_data['pred_pose']
        object_scores = frame_data['scores']
        
        for object_idx in range(len(object_centers)):
                if object_scores[object_idx] < 0.4:
                        continue
                
                bbox3D = np.concatenate([object_centers[object_idx], object_dimensions[object_idx]])
                color = [c/255.0 for c in util.get_color(object_idx)]
                box_mesh = util.mesh_cuboid(bbox3D, object_rotations[object_idx], color=color)
                meshes.append(box_mesh)
                
        frames_data.append(meshes)

In [28]:
output_path = "res"
for i, file in enumerate(image_files):
    frame = cv.imread(os.path.join(images_path, file))
    
    if not frames_data[i]:
        util.imwrite(frame, os.path.join(output_path, str(i)+'_combine.jpg'))
    else:
        im_drawn_rgb, im_topdown, _ = vis.draw_scene_view(frame, K, frames_data[i], text=None, scale=frame.shape[0], blend_weight=0.5, blend_weight_overlay=0.85)
        # im_concat = np.concatenate((im_drawn_rgb, im_topdown), axis=1)      
        util.imwrite(im_drawn_rgb, os.path.join(output_path, str(i)+'_combine.jpg'))